In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM , GRU
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re

## Define the source data

In [2]:
text_data = [
    "hello how are you" ,
    "hello how have you been" ,
    "hi there" ,
    "good morning" ,
    "good night" ,
    "have a nice day" ,
    "how is it going" ,
    "how have you been" , 
    "nice to meet you" ,
    "thank you"
]

### Create the tokenizer

In [3]:
#prepare the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [4]:
total_words

21

In [5]:
tokenizer.word_index

{'you': 1,
 'how': 2,
 'have': 3,
 'hello': 4,
 'been': 5,
 'good': 6,
 'nice': 7,
 'are': 8,
 'hi': 9,
 'there': 10,
 'morning': 11,
 'night': 12,
 'a': 13,
 'day': 14,
 'is': 15,
 'it': 16,
 'going': 17,
 'to': 18,
 'meet': 19,
 'thank': 20}

#### Prepare the input sequences

In [6]:
#create the sequences of word
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
input_sequences

[[4, 2],
 [4, 2, 8],
 [4, 2, 8, 1],
 [4, 2],
 [4, 2, 3],
 [4, 2, 3, 1],
 [4, 2, 3, 1, 5],
 [9, 10],
 [6, 11],
 [6, 12],
 [3, 13],
 [3, 13, 7],
 [3, 13, 7, 14],
 [2, 15],
 [2, 15, 16],
 [2, 15, 16, 17],
 [2, 3],
 [2, 3, 1],
 [2, 3, 1, 5],
 [7, 18],
 [7, 18, 19],
 [7, 18, 19, 1],
 [20, 1]]

### Padding of sequences

In [8]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences= pad_sequences(
    input_sequences , maxlen=max_sequence_len , padding='pre'
)

### Split into features and labels

In [9]:
import keras

In [10]:
input_sequences = np.array(input_sequences)
X,y = input_sequences[: , :-1] , input_sequences[:,-1]
y = keras.utils.to_categorical(y , num_classes = total_words)

In [11]:
input_sequences[: , -1]

array([ 2,  8,  1,  2,  3,  1,  5, 10, 11, 12, 13,  7, 14, 15, 16, 17,  3,
        1,  5, 18, 19,  1,  1])

In [12]:
y

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0.,

### Build the model

In [13]:
model = Sequential()
model.add(Embedding(total_words , 10 , input_length=max_sequence_len-1))
model.add(LSTM(150 , return_sequences=True))
model.add(LSTM(100))
model.add(Dense(total_words , activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 10)             210       
                                                                 
 lstm (LSTM)                 (None, 4, 150)            96600     
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 21)                2121      
                                                                 
Total params: 199331 (778.64 KB)
Trainable params: 199331 (778.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.compile( loss='categorical_crossentropy',optimizer='adam', metrics='accuracy')

In [22]:
model.fit(X, y, epochs=200, batch_size=1)

Epoch 1/200


23/23 [==============================] - 4s 9ms/step - loss: 3.0439 - accuracy: 0.1739
Epoch 2/200
23/23 [==============================] - 0s 8ms/step - loss: 2.9981 - accuracy: 0.2174
Epoch 3/200
23/23 [==============================] - 0s 9ms/step - loss: 2.8402 - accuracy: 0.2174
Epoch 4/200
23/23 [==============================] - 0s 8ms/step - loss: 2.7846 - accuracy: 0.2174
Epoch 5/200
23/23 [==============================] - 0s 9ms/step - loss: 2.7088 - accuracy: 0.2174
Epoch 6/200
23/23 [==============================] - 0s 8ms/step - loss: 2.6694 - accuracy: 0.2174
Epoch 7/200
23/23 [==============================] - 0s 8ms/step - loss: 2.6626 - accuracy: 0.2174
Epoch 8/200
23/23 [==============================] - 0s 7ms/step - loss: 2.6763 - accuracy: 0.2174
Epoch 9/200
23/23 [==============================] - 0s 9ms/step - loss: 2.6534 - accuracy: 0.2174
Epoch 10/200
23/23 [==============================] - 0s 8ms/step - loss: 2.5985 - accuracy: 0.2174
Epoch 1

#### Prediction on new data

In [24]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list],maxlen = max_sequence_len - 1 ,padding='pre')
    predicted = model.predict(token_list , verbose=0)
    predicted_word_index = np.argmax(predicted)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            output_word = word
            break
    return output_word                                  

In [35]:
new = "nice"

next_word = predict_next_word(model, tokenizer, new, max_sequence_len)
print(f"Next Word: {next_word}")

Next Word: to
